In [2]:
import cv2
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os
from keras.preprocessing.image import ImageDataGenerator

In [3]:
NUM_CLASSES = 2

In [28]:
cap = cv2.VideoCapture(0) 
cap.set(3, 640)
cap.set(4, 480) 
cap.set(10, 100) 

folder_name = 'thumbs-up'

folder_path = f'input/myImages3/{folder_name}/'

if os.listdir(folder_path):
    maxnum = max([int(file.split('frame')[1].split('.jpg')[0]) for file in os.listdir(folder_path)])
    i = maxnum
else:
    i = 0

while True: 
    i += 1

    success, img = cap.read()

    cv2.imwrite(folder_path+f'myImages3_{folder_name}_frame{i}.jpg', img)

    cv2.imshow('python', img)   
    
    if cv2.waitKey(20) == 27:  
        break

cv2.destroyWindow("python")  
cap.release()  
cv2.waitKey(1)


-1

In [35]:
input_shape = (28, 28, 1) 


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu')) 
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    'input/allImages',
    target_size=(28, 28),
    batch_size=128,
    shuffle=True,
    color_mode='grayscale', 
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    'input/valImages',
    target_size=(28, 28),
    batch_size=128,
    shuffle=True,
    color_mode='grayscale', 
    class_mode='categorical')  

model.fit(
    train_generator,
    epochs=2,  
    steps_per_epoch=len(train_generator),
    verbose=1,
    validation_data=validation_generator)

model.save('trained_gesture_model27.h5')

Found 11505 images belonging to 2 classes.
Found 2875 images belonging to 2 classes.
Epoch 1/2
90/90 [==============================] - 36s 388ms/step - loss: 0.6091 - accuracy: 0.6614 - val_loss: 0.4681 - val_accuracy: 0.7663
Epoch 2/2
90/90 [==============================] - 34s 379ms/step - loss: 0.3829 - accuracy: 0.8229 - val_loss: 0.2669 - val_accuracy: 0.8897


In [39]:
model = load_model('trained_gesture_model27.h5')

cap = cv2.VideoCapture(0) 
cap.set(3, 640) 
cap.set(4, 480)
cap.set(10, 100)


text = ['hi', 'thumbs-up']
positions = [(50, 50), (50, 100), (50, 150)] 
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
thickness = 3

image_width, image_height = 28, 28

font_color_red = (0, 0, 255) 
font_color_blue = (255, 0, 0) 


while True: 
    success, img = cap.read()

    resized_img = cv2.resize(img, (image_width, image_height)) 
    grey_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY) / 255

    pred = model.predict(np.array([grey_img] ))

    print(pred)

    num = int(np.argmax(pred))

    for i in range(NUM_CLASSES):
        if round(float(pred[:,i]), 3) > 0.5:
            cv2.putText(img, text[i]+f' {round(float(pred[:,i]), 3)}', positions[i], font, font_scale, font_color_red, thickness)
        else:
            cv2.putText(img, text[i]+f' {round(float(pred[:,i]), 3)}', positions[i], font, font_scale, font_color_blue, thickness)
    
    cv2.imshow('python', img) 
    
    if cv2.waitKey(20) == 27: 
        break

cv2.destroyWindow("python")  
cap.release()  
cv2.waitKey(1)


1/1 [==============================] - 0s 163ms/step
[[0.6501958  0.34980422]]
1/1 [==============================] - 0s 30ms/step
[[0.8351107  0.16488928]]
1/1 [==============================] - 0s 26ms/step
[[0.9681434  0.03185662]]
1/1 [==============================] - 0s 25ms/step
[[0.98342556 0.0165745 ]]
1/1 [==============================] - 0s 25ms/step
[[0.97422314 0.02577682]]
1/1 [==============================] - 0s 24ms/step
[[0.9740959  0.02590409]]
1/1 [==============================] - 0s 25ms/step
[[0.9724912  0.02750878]]
1/1 [==============================] - 0s 25ms/step
[[0.97087383 0.02912613]]
1/1 [==============================] - 0s 30ms/step
[[0.96003926 0.03996075]]
1/1 [==============================] - 0s 25ms/step
[[0.9326413  0.06735868]]
1/1 [==============================] - 0s 24ms/step
[[0.91670823 0.08329184]]
1/1 [==============================] - 0s 24ms/step
[[0.7360212 0.2639788]]
1/1 [==============================] - 0s 24ms/step
[[0.34991342 

-1